In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from typing import List, Tuple
from pytorch3d.renderer.cameras import get_screen_to_ndc_transform, get_ndc_to_screen_transform
from src.util.cameras import PatchPerspectiveCameras, get_patch_ndc_to_ndc_transform, get_ndc_to_patch_ndc_transform

In [2]:
# repeat sample image batch_size times with different patches of different sizes
image_size = [(256, 256), (128, 64), (64, 128), (32, 32)]
patch_size = [(64, 64), (32, 32), (16, 16), (8, 8)]

# wrt the center of the full image in pixels
patch_center = [(0, 0), (0, 0), (0, 0), (0, 0)]

cam_kwargs = {
    "znear": 0.0,
    "zfar": 80.0,
    "focal_length": 10.0,
    "principal_point": ((0.0, 0.0),),
    "R": torch.eye(3).unsqueeze(0),
    "T": torch.zeros(1, 3),
    "device": "cpu",
    "in_ndc": False,
    "image_size": image_size
    }

cam = PatchPerspectiveCameras(**cam_kwargs)

In [3]:
len(cam.get_ndc_camera_transform(**cam_kwargs))

4

In [4]:
world_to_patch_ndc_transform = cam.get_patch_projection_transform(patch_size, patch_center, **cam_kwargs)

In [5]:
patch_ndc_to_world_transform = world_to_patch_ndc_transform.inverse()

In [6]:
X_patch_ndc = torch.tensor([[0.5, 0.5, 1.0], [0.25, 0.25, 1.0], [0.75, 0.75, 1.0], [1., 1., 1.0]])
X_world = patch_ndc_to_world_transform.transform_points(X_patch_ndc)

In [7]:
world_to_patch_ndc_transform = cam.get_patch_projection_transform(patch_size, patch_center, **cam_kwargs)

In [8]:
X_patch_ndc = world_to_patch_ndc_transform.transform_points(X_world)  

In [9]:
# repeat sample image batch_size times with different patches of different sizes
image_size = [(256, 256)]
patch_size = [(128, 128)]
# wrt corner of the full image in pixels
patch_center = [(64, 64)]

znear = 0.0
zfar = 80.0
fov = np.deg2rad(60.0)

focal_length = image_size[0][0] / (2.0 * np.tan(fov / 2.0)) # focal length in pixels

cam_kwargs = {
    "znear": znear,
    "zfar": zfar,
    "focal_length": focal_length,
    "principal_point": ((0.0, 0.0),),
    "R": torch.eye(3).unsqueeze(0),
    "T": torch.zeros(1, 3),
    "device": "cpu",
    "in_ndc": False,
    "image_size": image_size
    }

cam = PatchPerspectiveCameras(**cam_kwargs)

In [10]:
# # topleft, topcenter, topright, midleft, midcenter, midright, botleft, botcenter, botright (-1,-1, 1) to (1, 1, 1)
# patch_ndc_points = [(-1.0, -1.0, 1.0),
#                     (0.0, -1.0, 1.0),
#                     (1.0, -1.0, 1.0),
#                     (-1.0, 0.0, 1.0),
#                     (0.0, 0.0, 1.0),
#                     (1.0, 0.0, 1.0),
#                     (-1.0, 1.0, 1.0),
#                     (0.0, 1.0, 1.0),
#                     (1.0, 1.0, 1.0)]

patch_ndc_points = [
    (-1.0, -1.0, 10.0),
    (1.0, 1.0, 60.0),
    (0.0, 0.0, 20.0),
    (-0.5, -0.5, 40.0),
    ]

expected_ndc = [
    (0.0, 0.0, 10.0),
    (1.0, 1.0, 60.0),
    (0.5, 0.5, 20.0),
    (0.25, 0.25, 40.0),
    ]

expected_screen = [
    (128.0, 128.0, 10.0),
    (0.0, 0.0, 60.0),
    (64, 64, 20.0),
    (96, 96, 40.0),
    ]


In [11]:
patch_ndc_to_ndc_transform = get_patch_ndc_to_ndc_transform(cameras=cam, 
                                                            image_size=image_size, 
                                                            patch_size=patch_size, 
                                                            patch_center=patch_center)
ndc_points = patch_ndc_to_ndc_transform.transform_points(torch.tensor(patch_ndc_points))

# patch_ndc_to_ndc_transform.matrix
# ndc_points[..., -1] = -((ndc_points[..., -1] + 1.0)/2.0 * (znear-zfar) + znear)  


ndc_points
# patch_ndc_to_ndc_transform.get_matrix()

tensor([[-1.0000, -1.0000, 10.0000],
        [ 0.0000,  0.0000, 60.0000],
        [-0.5000, -0.5000, 20.0000],
        [-0.7500, -0.7500, 40.0000]])

In [12]:
patch_ndc_revert_transform = get_ndc_to_patch_ndc_transform(cameras=cam,
                                                            image_size=image_size, 
                                                            patch_size=patch_size, 
                                                            patch_center=patch_center)
patch_ndc_points_revert = patch_ndc_revert_transform.transform_points(ndc_points)
patch_ndc_points_revert

tensor([[-1.0000, -1.0000, 10.0000],
        [ 1.0000,  1.0000, 60.0000],
        [ 0.0000,  0.0000, 20.0000],
        [-0.5000, -0.5000, 40.0000]])

In [13]:
ndc_to_screen_transform = get_ndc_to_screen_transform(cameras=cam, image_size=image_size, with_xyflip=True)
# For some reason NDC to screen does wierd stuff. Would assume they do lower right corner instead of upper left
screen_points = ndc_to_screen_transform.transform_points(ndc_points)
screen_points



tensor([[256., 256.,  10.],
        [128., 128.,  60.],
        [192., 192.,  20.],
        [224., 224.,  40.]])

In [14]:
world_points = cam.unproject_points(screen_points) # screen --> world
world_points

tensor([[11.5470, 11.5470, 10.0000],
        [34.6410, 34.6410, 60.0000],
        [17.3205, 17.3205, 20.0000],
        [40.4145, 40.4145, 40.0000]])

In [15]:
screen_points_revert = cam.transform_points_screen(world_points)
screen_points_revert

tensor([[-2.5600e+02, -2.5600e+02,  1.0000e-01],
        [-1.2800e+02, -1.2800e+02,  1.6667e-02],
        [-1.9200e+02, -1.9200e+02,  5.0000e-02],
        [-2.2400e+02, -2.2400e+02,  2.5000e-02]])

In [16]:
ndc_points_revert = cam.transform_points_ndc(world_points)
ndc_points_revert

tensor([[3.0000, 3.0000, 0.1000],
        [2.0000, 2.0000, 0.0167],
        [2.5000, 2.5000, 0.0500],
        [2.7500, 2.7500, 0.0250]])